## Image Analytics Project 3: Classifying land cover using remote sensing data from Microsoft Planetary Computer (MPC)
### By Mira Rayson
### For James Rapaport

This code performs a series of operations to classify land cover using remote sensing data from the Microsoft Planetary Computer. The process begins by connecting to the Planetary Computer's STAC API to retrieve a specific Landsat 8 image. It then defines a geographical bounding box to extract a subset of the image, creating a windowed view of the data. The script samples the image's spectral bands at specified coordinates to generate training data, which is used to train a support vector classifier (SVC). The trained model is then applied to classify the entire subset of the image, generating a classified raster. The results are saved in a GeoTIFF format, with additional steps to apply pseudo-coloring based on predefined land cover classes. Finally, the code counts and prints the number of pixels for each classified land cover type in both the newly created classification and an existing land cover classification for comparison. This workflow integrates data retrieval, preprocessing, machine learning classification, and result visualization for land cover analysis.

In [1]:
# Importing necessary packages
import numpy
import pystac_client
import planetary_computer
import shapely
import shapely.ops
import fiona
import sqlite3
import rasterio
from rasterio.plot import reshape_as_image
from rasterio.enums import ColorInterp 
from rasterio.windows import Window
from rasterio import features
from rasterio.crs import CRS
from rasterio.windows import Window
from rasterio.windows import from_bounds
from rasterio.windows import  round_window_to_full_blocks
from time import time
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from matplotlib import pyplot
from datetime import datetime

### Making connection to planetary computer and importing chosen data

In [2]:
client = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
)
landsat_id = 'LC08_L2SP_008029_20201016_02_T1'
search = client.search(
    collections=['landsat-c2-l2'],
    ids=[landsat_id]
)
item = search.item_collection()[0]
bands = ['blue', 'green', 'red', 'nir08', 'swir16', 'swir22']
features = len(bands)

right = 472460
left = 422400
top = 4966160
bottom = 4917440

bbox = (left, bottom, right, top)

### Defining the area of interest and creating a window

In [3]:
with rasterio.open(item.assets['blue'].href, 'r') as input:
    window=rasterio.windows.from_bounds(*bbox, input.transform)
    window=rasterio.windows.round_window_to_full_blocks(
                window,
                [(input.profile['blockysize'], input.profile['blockxsize'])])
  
    data=input.read(1, window=window)
    
    profile=input.profile
    ulx, uly=input.xy(int(window.row_off), int(window.col_off), offset='ul')
    profile['transform']=rasterio.Affine(
        input.transform.a,
        input.transform.b,
        ulx,
        input.transform.d,
        input.transform.e,
        uly
    )
    profile['compress']='deflate'
    profile['width']=int(window.width)
    profile['height']=int(window.height)
    profile['driver']='COG'
    profile['count']=1
    profile['dtype']=rasterio.uint8


### Sampling the Data

In [4]:
with fiona.open('training.gpkg', layer='training-2020') as training:
    y_labels=numpy.fromiter([l['properties']['code'] for l in training], numpy.uint8)
    coordinates=[c['geometry']['coordinates'] for c in training]
    X_samples=numpy.empty((len(coordinates), features), dtype=numpy.uint16)
 
    for b in bands:
        with rasterio.open(item.assets[b].href) as image:
            samples=image.sample(coordinates)
            samples=numpy.fromiter([s[0] for s in samples],
                                     numpy.uint16).reshape(len(y_labels), 1)
            numpy.put_along_axis(X_samples,
                                 numpy.full_like(samples, bands.index(b)),
                                 samples, axis=1)

### Printing accuracy assessments and completing classification

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X_samples, y_labels, test_size=0.7, stratify=y_labels)

classification = SVC()
classification.fit(X_train, y_train)

classification_test = classification.predict(X_test)

print('Accuracy Score:')
print(f'{accuracy_score(y_test, classification_test)}')
print('Classification Report:')
print(classification_report(y_test, classification_test, zero_division=1))
print('Confusion Matrix:')
print(confusion_matrix(y_test, classification_test))

with (rasterio.open(f'{landsat_id}_classified_svc_nrcan.tif',
                   mode='w',
                   **profile) as classified):
    data = numpy.empty((window.width * window.height, features), dtype=numpy.uint16)
    for b in bands:
        with rasterio.open(item.assets[b].href) as band:
            data_band = reshape_as_image(
                            band.read(window=window)).reshape(-1, 1)
            numpy.put_along_axis(data, 
                                numpy.full_like(data_band, bands.index(b)), 
                                data_band, axis=1)
    prediction = classification.predict(data).reshape(int(window.height), 
                                                      int(window.width))
    
    classified.write(prediction, indexes=1)

Accuracy Score:
0.7380952380952381
Classification Report:
              precision    recall  f1-score   support

           1       0.55      0.86      0.67         7
           5       0.75      0.86      0.80         7
           6       0.50      0.14      0.22         7
          10       0.67      0.86      0.75         7
          17       1.00      0.71      0.83         7
          18       1.00      1.00      1.00         7

    accuracy                           0.74        42
   macro avg       0.74      0.74      0.71        42
weighted avg       0.74      0.74      0.71        42

Confusion Matrix:
[[6 0 1 0 0 0]
 [0 6 0 1 0 0]
 [5 0 1 1 0 0]
 [0 1 0 6 0 0]
 [0 1 0 1 5 0]
 [0 0 0 0 0 7]]


### Applying Pseudo Colour


In [6]:
with sqlite3.connect('training.gpkg') as db:
    colours={}
    cursor=db.cursor()
    cursor.execute(
        """SELECT code, red, green, blue FROM landcover_class"""
    )
    for r in cursor.fetchall():
        colours[r[0]] = [r[1], r[2], r[3]]

prediction = classification.predict(data).reshape(int(window.height), int(window.width))

with rasterio.open(f'{landsat_id}_classified.tif', mode='w', **profile) as classified:
    classified.write(prediction, indexes=1)
    classified.write_colormap(1, colours)

### Counting number of pixels in each class for both classifications

In [7]:
landcover2020_subset = 'landcover-2020-classification-subset.tif'
with rasterio.open(landcover2020_subset) as src:
    classification_data=src.read(1)
    unique_classes, class_counts=numpy.unique(classification_data, return_counts=True)
for class_value, count in zip(unique_classes, class_counts):
    print(f"Class {class_value}: {count} pixels")

landsat_subset = 'LC08_L2SP_008029_20201016_02_T1_classified.tif'
with rasterio.open(landsat_subset) as src2:
    classification_data2=src2.read(1)
    unique_classes2, class_counts2=numpy.unique(classification_data2, return_counts=True)
for class_value2, count2 in zip(unique_classes2, class_counts2):
    print(f"Class {class_value2}: {count2} pixels")

Class 0: 168633 pixels
Class 1: 541793 pixels
Class 5: 303600 pixels
Class 6: 647138 pixels
Class 8: 309 pixels
Class 10: 89181 pixels
Class 14: 1009 pixels
Class 15: 1455 pixels
Class 16: 2611 pixels
Class 17: 233254 pixels
Class 18: 1222281 pixels
Class 1: 898572 pixels
Class 5: 289327 pixels
Class 6: 231978 pixels
Class 10: 360374 pixels
Class 17: 75427 pixels
Class 18: 1355586 pixels
